In [1]:
import torch
from sentence_transformers import SentenceTransformer, util
from transformers import (pipeline, AutoTokenizer, AutoModelForCausalLM)

In [2]:
category_descriptions = {
    'Библиотека':'Материалы об исторических событиях и жизни в прошлом, особенно в контексте военных лет, Петрограда, солдат и хронологии начала XX века.',
    'Россия':'Новости и события, связанные с внутренней политикой, обществом, заявлениями официальных лиц и жизнью в российских городах, включая Москву.',
    'Мир':'Международная повестка: события за рубежом, политика США, мировая экономика, конфликты и международные отношения.',
    'Экономика':'Вопросы, касающиеся российского и глобального бизнеса, рубля, доллара, цен, инфляции,ВВП а также деятельности компаний и рыночных трендов.',
    'Интернет и СМИ':'Темы, связанные с интернетом, СМИ, цифровыми технологиями и крупными IT-компаниями (например, Microsoft), а также новостными порталами.',
    'Спорт':'События из мира спорта: матчи, турниры, чемпионаты, сборные команды и результаты соревнований.',
    'Культура':'Кино, театр, музыка, роли актеров и режиссёров. Анонсы и обсуждения культурных событий и персоналий.',
    'Наука и техника':'Новости науки и технологий, исследования, космос (МКС, NASA), разработки и мнения учёных.',
    'Бизнес':'Экономическая активность компаний, инвестиции, отчетность, валютные потоки, изменения в бизнес-среде.',
    'Из жизни':'Лайфстайл, интересные события, происшествия, человеческие истории и нестандартные новости.',
    'Бывший СССР':'Политические и социальные события в странах бывшего Советского Союза: Украина, Молдова, Киргизия и другие',
    'Дом':'Недвижимость, строительство, стоимость жилья, квадратные метры, компании-застройщики и рынок жилья.',
    'Путешествия':'Темы туризма, поездок, путешествий внутри России и за границу, статистика и потребительское поведение туристов.',
    'Силовые структуры':'Работа МВД, судов, следственных органов, дела против чиновников, правонарушения и силовая политика.',
    'Ценности':'Мода, бренды, коллекции одежды, часы, товары класса люкс и потребительское поведение в этой сфере.',
    'Легпром':'Новости лёгкой промышленности, производства текстиля и одежды, статистика Минпромторга и экспортно-импортная динамика.',
    'Культпросвет':'Деятельность Минкультуры, общественные и культурно-просветительские инициативы, высказывания чиновников.',
    'Крым':'События в Крыму и Севастополе, деятельность местных властей, политика интеграции и региональное управление на полуострове.',
    '69-я параллель':'Жизнь и события в Мурманской области и других северных регионах, арктическая повестка, бюджет и развитие инфраструктуры.'
}

In [7]:
import gc
torch.cuda.empty_cache()
gc.collect()

23031

In [6]:
def load_models():
    dense_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
    descs = list(category_descriptions.values())
    desc_embeddings = dense_model.encode(descs, convert_to_tensor=True, normalize_embeddings=True)

    llm_model = AutoModelForCausalLM.from_pretrained(
        "t-tech/T-lite-it-1.0",
        torch_dtype=torch.float16,
        device_map="auto"
    )
    tokenizer = AutoTokenizer.from_pretrained("t-tech/T-lite-it-1.0")
    generator = pipeline("text-generation", model=llm_model, tokenizer=tokenizer)

    return dense_model, desc_embeddings, generator

dense_model, desc_embeddings, generator = load_models()
index_to_class = list(category_descriptions.keys())
descriptions = list(category_descriptions.values())

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

OSError: Файл подкачки слишком мал для завершения операции. (os error 1455)

In [8]:
def predict_with_llm(text, top_k=5):
    query_embedding = dense_model.encode(text, convert_to_tensor=True, normalize_embeddings=True)
    scores = util.cos_sim(query_embedding, desc_embeddings)[0]
    top_indices = torch.topk(scores, k=top_k).indices
    candidates = [(index_to_class[i], descriptions[i]) for i in top_indices]

    prompt = f"Определи, к какой из следующих категорий относится текст:\n\n"
    prompt += f"Текст: \"{text}\"\n\nКатегории:\n"
    for i, (label, desc) in enumerate(candidates, 1):
        prompt += f"{i}. {label} — {desc}\n"
    prompt += "\nВыбери номер подходящей категории и укажи только её название.\nОтвет:"


    output = generator(
        prompt,
        max_new_tokens=20,
        temperature=0.3,
        top_p=0.9,
        do_sample=False,
        return_full_text=False
    )

    result = output[0]["generated_text"].strip()

    for label, desc in candidates:
        if label.lower() in result.lower():
            return desc

    return candidates[0][1]

In [9]:
example = """Президент России Владимир Путин прибыл в Волгоград ранним утром 1 января 2014 года, сообщает РИА Новости.
Путин провел совещание по борьбе с терроризмом, возложил цветы на месте, где 30 декабря был взорван троллейбус,
а затем навестил в больнице пострадавших при волгоградских терактах. Поездка не была анонсирована заранее.
В антитеррористическом совещании приняли участие главы ФСБ и МВД Александр Бортников и Владимир Колокольцев,
начальник ГУ МВД РФ по Волгоградской области Александр Кравченко, начальник УФСБ по области Сергей Блинов,
министр здравоохранения России Вероника Скворцова и губернатор региона Сергей Боженов."""
print(predict_with_llm(example))

Новости и события, связанные с внутренней политикой, обществом, заявлениями официальных лиц и жизнью в российских городах, включая Москву.
